# 7.a

In [1]:
#Importing the libraries
import numpy as np
import pandas as pd

In [2]:
#Loading the dataset and preprocessing
df = pd.read_excel("../Dataset/Input_Dataset/DDW-C17-0100.XLSX", header = [2])
#Renaming the column name
df = df.rename(columns = {"State" : "State code", "Unnamed: 3" : "Language", "Unnamed: 4" : "Count", "State name" : "State"})
df = df.iloc[:, [0, 1, 3, 4]]
#Droping the unimportant columns
df.drop([0, 1, 2], axis = 0, inplace = True)
df.dropna(axis=0, inplace=True)
#Sorting the dataframe
df.sort_values(by = "Count", inplace = True, ascending=False)
df_main = df.copy()

In [3]:
#Loading all the dataset and preprocessing
for i in range(2, 36):
    if i < 10:
        file_path = r"../Dataset/Input_Dataset/DDW-C17-0" + str(i) + "00.XLSX"
    else:
        file_path = r"../Dataset/Input_Dataset/DDW-C17-" + str(i) + "00.XLSX"
    df = pd.read_excel(file_path, header = [2])
    #Renamin the column name
    df = df.rename(columns = {"State" : "State code", "Unnamed: 3" : "Language", "Unnamed: 4" : "Count", "State name" : "State"})
    df = df.iloc[:, [0, 1, 3, 4]]
    #Droping the unimportant columns
    df.drop([0, 1, 2], axis = 0, inplace = True)
    df.dropna(axis=0, inplace=True)
    df_main = df_main.append(df)

In [4]:
#Initializing the regions
regions = {"North" :["PUNJAB", "HIMACHAL PRADESH", "HARYANA", "NCT OF DELHI", "CHANDIGARH", "UTTARAKHAND", "JAMMU & KASHMIR"],
          "West": ["RAJASTHAN", "GUJARAT", "MAHARASHTRA", "GOA", "DAMAN & DIU", "DADRA & NAGAR HAVELI"], 
          "Central" : ["UTTAR PRADESH", "MADHYA PRADESH", "CHHATTISGARH"], 
          "East" : ["BIHAR", "WEST BENGAL", "ODISHA", "JHARKHAND"], 
          "South" : ["KARNATAKA", "PUDUCHERRY", "LAKSHADWEEP", "KERALA", "TAMIL NADU", "ANDHRA PRADESH"], 
          "North-East" : ["ASSAM", "SIKKIM", "MEGHALAYA", "TRIPURA", "ARUNACHAL PRADESH", "MANIPUR", "NAGALAND", "MIZORAM", "ANDAMAN & NICOBAR ISLANDS"]}

In [5]:
df_main["region"] = ""

In [6]:
#Defining the regions to each state
for key, value in regions.items():
    df_main.loc[df_main.State.isin(value), "region"] = key

In [7]:
#Droping the unimportant columns
df_main.drop(["State code", "State"], axis=1, inplace=True)

In [8]:
#Making groupby using region and language
gb = df_main.groupby(by = ["region", "Language"]).sum()

In [9]:
gb

Count
region  Language                     
Central ADI                       510
        AFGHANI/KABULI/PASHTO     286
        ANAL                       22
        ANGAMI                     46
        AO                         56
...                               ...
West    WANCHO                      4
        YIMCHUNGRE                 29
        ZELIANG                     3
        ZEMI                       72
        ZOU                         9

[707 rows x 1 columns]

In [10]:
#Sorting the dataframe based on count
gb.sort_values(by = "Count", inplace=True, ascending=False)

In [11]:
#finding the top 3 languages for each region
main_list = []
for key in regions.keys():
    temp_list = []
    temp_list = list(gb.loc[key].index)[:3]
    temp_list.insert(0, key)
    main_list.append(temp_list)

In [12]:
#Making the final dataframe and sorting it based on first column
df_final = pd.DataFrame(main_list, columns=["region", "language-1", "language-2", "language-3"])
df_final.sort_values("region", inplace=True)

In [13]:
df_final

,region,language-1,language-2,language-3
2,Central,HINDI,URDU,BHILI/BHILODI
3,East,HINDI,BENGALI,ODIA
0,North,HINDI,PUNJABI,KASHMIRI
5,North-East,ASSAMESE,BENGALI,HINDI
4,South,TELUGU,TAMIL,KANNADA
1,West,HINDI,MARATHI,GUJARATI


In [14]:
#Storing the final result
df_final.to_csv("../Dataset/Output_Dataset/region-india-a.csv", index=False)

# 7.b

In [15]:
#Loading the dataset and preprocessing
df = pd.read_excel("../Dataset/Input_Dataset/DDW-C17-0100.XLSX", header = [2])
df = df.iloc[3:, [0, 1, 3, 4, 8, 9, 13, 14]]
#Renaming the column name
df.columns = ["State Code", "Name", "Language1", "Count1", "Language2", "Count2", "Language3", "Count3"]
#Making the copy of original dataframe
df_main = df.copy()

In [16]:
#Loading the remaining dataset and preprocessing
for i in range(2, 36):
    if i < 10:
        file_path = r"../Dataset/Input_Dataset/DDW-C17-0" + str(i) + "00.XLSX"
    else:
        file_path = r"../Dataset/Input_Dataset/DDW-C17-" + str(i) + "00.XLSX"
    df = pd.read_excel(file_path, header = [2])
    df = df.iloc[3:, [0, 1, 3, 4, 8, 9, 13, 14]]
    #Renaming the column name
    df.columns = ["State Code", "Name", "Language1", "Count1", "Language2", "Count2", "Language3", "Count3"]\
    #Adding it to final dataset
    df_main = df_main.append(df)

In [17]:
df_main["region"] = ""

In [18]:
#Defining the region for each state
for key, value in regions.items():
    df_main.loc[df_main.Name.isin(value), "region"] = key

In [19]:
#Droping the unimportant datset
df_main.drop(["State Code", "Name"], axis=1, inplace=True)

In [20]:
#Finding the unique language
unique_language = set(list(df_main["Language1"].unique())).union(set(list(df_main["Language2"].unique()))).union(set(list(df_main["Language3"].unique())))

In [21]:
#Creating a new dictonary
dict = {"Region" : [],
       "Language" : [], 
       "Count" : []}

In [22]:
#Calculating the total population speaking each language
for st in regions.keys():
    for lan in unique_language:
        if lan == "nan":
            continue
        count = 0
        #Adding the population who speak lan as first choice
        count = count + np.sum(df_main[(df_main["region"] == st) & (df_main["Language1"] == lan)]["Count1"])
        #Adding the population who speak lan as second choice
        count = count + np.sum(df_main[(df_main["region"] == st) & (df_main["Language2"] == lan)]["Count2"])
        #Adding the population who speak lan as thirsd choice
        count = count + np.sum(df_main[(df_main["region"] == st) & (df_main["Language3"] == lan)]["Count3"])
        if count == 0:
            continue
        dict["Region"].append(st)
        dict["Language"].append(lan)
        dict["Count"].append(count)

In [23]:
df_final = pd.DataFrame(dict)

In [24]:
#Finding the total number of population speaking particilar language in each region
gb = df_final.groupby(["Region", "Language"]).sum()

In [25]:
gb.sort_values(by = "Count", inplace=True, ascending=False)

In [26]:
#Finding the top 3 language for each region
main_list = []
for key in regions.keys():
    temp_list = []
    temp_list = list(gb.loc[key].index)[:3]
    temp_list.insert(0, key)
    main_list.append(temp_list)

In [27]:
#Making the dataframe and sorting it
df_final = pd.DataFrame(main_list, columns=["region", "language-1", "language-2", "language-3"])
df_final.sort_values("region", inplace=True)

In [28]:
df_final

,region,language-1,language-2,language-3
2,Central,HINDI,ENGLISH,URDU
3,East,HINDI,BENGALI,ODIA
0,North,HINDI,PUNJABI,ENGLISH
5,North-East,ASSAMESE,BENGALI,HINDI
4,South,TELUGU,TAMIL,KANNADA
1,West,HINDI,MARATHI,GUJARATI


In [29]:
#Storing the final result
df_final.to_csv("../Dataset/Output_Dataset/region-india-b.csv", index=False)